# Necessary Modules

In [71]:
# If you are planning to run matplotlib qt, check if you have the following packages; if not, run this cell.
#!pip install scikit-image
#!pip install scikit-learn
#!pip install photutils
#!pip install specutils
#!pip install ccdproc
#!pip install PyQt6
#!pip install PySide6
#!pip install PyQt5

In [2]:
# Import the following:
from pyvista import imred, tv, spectra, stars, slitmask, image
import numpy as np
import pdb
import copy
import matplotlib.pyplot as plt
import os
from astropy.table import vstack
import pandas as pd
import importlib
from astropy.io import fits

# Directory setup

In [3]:
# put directory name with images here
indir='UT230909'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  UT230909/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [4]:
# Display the contente of the directory
red.log().show_in_notebook(display_length=5)

idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,Flat_SEG3G2.0001.fits,2023-09-09T01:31:47.843412,,6:56:00.00,75:00:00.00,2.0
1,Flat_SEG3G2.0002.fits,2023-09-09T01:33:24.544267,,6:56:00.00,75:00:00.00,1.0
2,Flat_SEG3G2.0003.fits,2023-09-09T01:34:47.647398,,6:56:00.00,75:00:00.00,0.5
3,Flat_SEG3R2.0004.fits,2023-09-09T01:38:24.858222,,6:56:00.00,75:00:00.00,0.5
4,Flat_EMPTY_TEST.0005.fits,2023-09-09T01:40:21.135840,,6:56:00.00,75:00:00.00,0.5
5,Flat_SEG3R1.0006.fits,2023-09-09T01:43:41.792202,,6:56:00.00,75:00:00.00,0.5
6,Flat_SEG3R1.0008.fits,2023-09-09T01:47:35.037138,,6:56:00.00,75:00:00.00,0.5
7,Flat_SEG3G2.0009.fits,2023-09-09T01:49:32.885458,,6:56:00.00,75:00:00.00,0.5
8,SEG3G2.0010.fits,2023-09-09T02:19:53.351408,Seg3G2-Mask,21:21:32.11,19:08:59.20,20.0
9,SEG3G2.0011.fits,2023-09-09T02:33:52.513049,Seg3G2-Mask,21:21:34.14,19:09:06.69,20.0


**Setup the TV window**

In [5]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
%matplotlib qt
t=tv.TV()
plotinter=True

# following lines only for fully non-interactive demo of notebook
#%matplotlib inline
#plotinter=False
#t=None

# Calibration images.

In [6]:
# Display bias or any calibration image you may want to look at.
bi = red.reduce(74)
if t is not None:
     t.tv(bi)

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


## Master bias.

In [7]:
biastims=[74,75,76,77,78] # Make a list of biases using file numbers.
bias=red.mkbias(biastims,display=None) # Make a master bias.

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0075.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0076.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0077.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0078.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [8]:
# Display master bias.
if t is not None:
    t.tv(bias)

## Master dark

In [9]:
darktims=[94,95,96] # Make a list of darks using file numbers.
dark=red.mkdark(darktims,display=None) # Make a master dark.

  Reading file: UT230909\dark.0094.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0095.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [11]:
# Display dark.
if t is not None:
    t.tv(dark)

## Master flat

In [12]:
flatims=[21,22] # Make a list of flats using file numbers.
flat=red.mkflat(flatims,spec=True) # Make masteer flat.

  Reading file: UT230909\Flat_SEG3G2.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [13]:
# Display flat
if t is not None:
    t.tv(flat)

# Science image reduction

In [18]:
# Reduce a using the calibration images previously defined.
star1=red.reduce(20, crbox='lacosmic', bias=bias, flat=flat, dark=dark, display=t)

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 
  subtracting bias...
  subtracting dark...
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
  flat fielding...


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:765: RuntimeWarning: divide by zero encountered in divide
  corr.data /= flat.data
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:766: RuntimeWarning: divide by zero encountered in divide
  corr.uncertainty.array /= flat.data
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:158: RuntimeWarning: invalid value encountered in multiply
  y *= step
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:168: RuntimeWarning: invalid value encountered in add
  y += start


  See flat-fielded image and original with - (minus) key.
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


# Find and display slits on a flat image

In [19]:
# Use a single flat
flat1 = red.reduce(22) # Reading a flat.

  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [41]:
trace=spectra.Trace(transpose=True) # Setup a trace base based on KOMOS (Trancepose = True)
t.tvclear()
bottom,top = trace.findslits(flat1,display=t,thresh=0.5,sn=True) # This does not take mkflat().

In [42]:
vars(trace) # visualize the trace

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([205.39570053,   6.70214955, -19.58890376], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([353.47561566,   6.31656815, -13.19157469], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([500.31128681,   6.02780702,  -9.3204817 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([725.38596237,   6.39430102,  -3.60916434], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([934.65809011,   6.48687832,   1.55820243], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1023.40503861,    6.46322524,    3.67504815], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1089.0796024,    6.4892661,    5.2677399], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1154.51935672,    6.50896277,    6.81061458], d

# Read slit mask from a kms-file

In [43]:
kmsfile='kms/Copy of kosmos.23.seg3g2.kms' # File directory.
targets = slitmask.read_kms(kmsfile,sort='YMM') # Reading file.
targets # Visualing file content.

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.87,-10.801
TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.61
TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272


## Convert the targets table to a pandas Data Frame

In [44]:
df = targets.to_pandas() # Convertion taking place.
df # A look at your table in panda formate.

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
0,TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
1,TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
2,TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
3,TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.870,-10.801
4,TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.610
5,TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
8,TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780


## Remove specified rows from Data Frame

In [45]:
# Specify the indices of the rows you want to remove
rows_to_remove = [0,1,2,3,4,5,8,10,12,14,15]
# Remove the specified rows
df_cleaned = df.drop(rows_to_remove)
df_cleaned # Visualize the results

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780
11,TARG104,NN,STRAIGHT,0.9,10.0,0.0,212136.755,190631.54,0.154,1.707,-11.247,25.194
13,TARG102,NN,STRAIGHT,0.9,10.0,0.0,212134.103,190555.71,0.154,1.707,-4.831,31.313


## Filter and process data based on index values

In [46]:
# Give your index value
in_dex = [6, 7, 9, 11, 13]

# Create a new list to store the filtered lines
filtered_rows = []

for index, line in enumerate(trace.rows):
    if index in in_dex:
        filtered_rows.append(line)

# Replace the original trace.rows with the filtered list
trace.rows = filtered_rows

trace.rows # Visulize the results.

[[1089, 1128], [1154, 1193], [1352, 1391], [1542, 1581], [1681, 1720]]

# Object tracing

In [47]:
# Create your own trace from scratch.
# Don't worry about assigning a value to sc0 because the default is 2028, the center of the KOSMOS lens.
trace1=spectra.Trace(lags=range(-150,150),
                    rows= trace.rows ,transpose=red.transpose, rad=5, degree= 3, sigdegree=3)
vars(trace1) # Visualize your own trace

{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 3,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': None,
 'sigmodel': None,
 'sc0': None}

## This cell configures and performs a trace operation on an image using predefined rows and parameters:

In [49]:
#Trace
srow = [] # list to allow for multiple tracing.
for lis in trace.rows:
    # Setting the center for each slit and adding it to the list.
    srow.append(lis[0]+((lis[1]-lis[0])//2))

# rad is setting the width of your trace. It will take the center position to be the star position given taking from DS9.
trace1.trace(star1,srow,skip=10,
                    gaussian = True, display=t, rad= 5) #star1
vars(trace1) # Visulize the trace information

C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:158: RuntimeWarning: invalid value encountered in multiply
  y *= step
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:168: RuntimeWarning: invalid value encountered in add
  y += start


  Tracing row: 1700
  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 3,
 'pix0': 0,
 'spectrum': array([    0.        , 27576.87109375,  1537.84667969, ...,
          445.91418457,  1486.26672363,   346.99194336]),
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': [<Polynomial1D(3, c0=1106.33525282, c1=-0.00049708, c2=0.00000084, c3=0.)>,
  <Polynomial1D(3, c0=1170.3889761, c1=-0.00158765, c2=0.00000094, c3=0.)>,
  <Polynomial1D(3, c0=1374.56562508, c1=-0.00685958, c2=0.00000231, c3=0.)>,
  <Polynomial1D(3, c0=1572.05485728, c1=-0.01197227, c2=0.00000349, c3=0.)>,
  <Polynomial1D(3, c0=1709.74053124, c1=-0.01559895, c2=0.00000434, c3=0.)>],
 'sigmodel': [<Polynomial1D(3, c0=3.86070337, c1=-0.00257707, c2=0.00000083, c3=-0.)>,
  <Polynomial1D(3, c0=3.9288116, c1=-0.00241408, c2=0.00000062, c3=-0.)>,
  <Polynomial1D(3, c0=3.71836246, c1=-0.00226784, c2=0.00000065, c3=-0.)>,
  <Polynomial1D(3, c0=3.

## Now extract the sciencee frame

In [50]:
out1 = trace1.extract2d(star1, display=t) # 2D Extraction using the reduced image

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


## Now extract the arc frame

In [51]:
arcs=red.sum([23,24]) # Make a master arc
# Display the lamp
if t is not None:
     t.clear()
     t.tv(arcs)

  Reading file: UT230909\Ne_120s_SEG3G2.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Ne_360s_SEG3G2.0024.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


In [53]:
arcec1=trace1.extract2d(arcs, display=t) # 2D Extraction using master arcs.

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


## This cell checks if the number of target objects matches the number of slits and updates FITS headers:

In [54]:
if len(targets) == len(bottom) : # Checking if the conditions is mate.
    for arc,target in zip(arcec1,targets) :
        arc.header['XMM'] = target['XMM']
        arc.header['YMM'] = target['YMM']
else :
    print('ERROR, number of identified slits does not match number of targets')

# This cell performs wavelength calibration for each arc spectrum in 'arcec1' using a predefined wavelength solution file:

In [55]:
for i,arc in enumerate(arcec1) :
    # get an initial guess at the shift from reference using XMM (KOSMOS red center!)
    wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
    nrow=arc.shape[0] # this is referring to the slit width of each slit. 
    
    shift=int(arc.header['XMM'])
    lags=np.arange(shift-400,shift+400)

    iter = True
    while iter :
        iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                            lags=lags,thresh=5,
                file='new_wave_lamp/copy_new_neon_red_center.dat',
                rad = 5) # The line computing identification.
        lags=np.arange(-19,19) # 
        plt.close()
        
    # Do the 2D wavelength solution, sampling 5 locations across slits.
    wav.identify(arc,plot=True,nskip=nrow//5,thresh=5)
    plt.close()

  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  -405 394
  Derived pixel shift from input wcal:  [-255.75714302]


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:696: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:   64.574 Angstroms (36 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:   64.574 Anstroms
  input from plot window...
  rms:    2.069 Anstroms
  input from plot window...
  rms:    0.079 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -19 18
  Derived pixel shift from input wcal:  [-2.5908971e-08]
  See identified lines.
  rms:    0.072 Angstroms (38 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    0.072 Anstroms
  input from plot wi

# This cell visualizes and processes spectral data after extraction, involving both plotting and adding wavelength information:

In [57]:
t.clear()
plt.figure()
for i,(o,a) in enumerate(zip(out1,arcec1)) :
    print(o.shape)
    print(a.wave)
    o.add_wave(a.wave) # Adding wavelenght information.
    name = o.header["FILE"].split(".")[0]
    print(name)
    #t.tv(o)
    #t.tv(o.wave)
    plt.plot(o.wave[19],o.data[19]) # 19 center of the slits, but could be any.
    #o.write(name + "_20_{:03d}.fits".format(i)) # If you wish to save the 2D spectra with wavelength.

(39, 4096)
[[9348.3564274  9347.33281452 9346.30913881 ... 5264.61887174
  5263.80159313 5262.98447824]
 [9348.41474189 9347.39104083 9346.36727706 ... 5264.58594208
  5263.7685527  5262.9513269 ]
 [9348.47073856 9347.44695313 9346.42310509 ... 5264.55485647
  5263.73736111 5262.92002922]
 ...
 [9348.99551347 9347.97112933 9346.94668336 ... 5264.59514997
  5263.77691065 5262.95883396]
 [9348.97038592 9347.94605093 9346.92165405 ... 5264.62860579
  5263.81042869 5262.99241428]
 [9348.94294053 9347.91865851 9346.89431452 ... 5264.66390565
  5263.84579557 5263.02784824]]
SEG3G2
(39, 4096)
[[9720.99221922 9719.96395204 9718.93562326 ... 5601.28025758
  5600.44343452 5599.60676667]
 [9720.9650616  9719.93681768 9718.90851215 ... 5601.23917771
  5600.40225971 5599.56549684]
 [9720.93910966 9719.91088841 9718.88260558 ... 5601.20064208
  5600.36363362 5599.52678024]
 ...
 [9720.774114   9719.74632507 9718.71847447 ... 5601.40424946
  5600.56683819 5599.72958167]
 [9720.79036032 9719.76257415 

# 1D extraction

In [63]:
importlib.reload(spectra)
def model(x) :
    return x*0.

list_1 = []
for lis in trace.rows:
    list_1.append(lis[0]) # Get the beginning value of each slit.
    
fig,(ax1, ax2)= plt.subplots(2,1)
for i in range(len(out1)) :
    nrow=out1[i].shape[0]//2 # Getting the center of slits.
    center = list_1[i]+nrow # The value for finding the value of the peak around the center of each 2D extracted spectra (out1)
    trace2 = spectra.Trace(transpose=False, sc0=center, degree=3, lags=[-19,19])  #  diff=10000,bundle=10000
    trace2.rows = [0,out1[i].data.shape[0]]
    trace2.index = [0]
    peak,ind = trace2.findpeak(out1[i],thresh=10,sort=True, width =19, percent =80, method= 'nearest', axis = 1,)# Set the sort value to variable to True in order to get the brightest peak value.
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace2.model = [model]
        spec,back_spec=trace2.extract(out1[i],rad=5,back= [[-15, -5], [15, 5]] , display=t,) # This line is important because you are extracting sky here.
        spec.wave = out1[i].wave[peak]
        back_spec.wave=out1[i].wave[peak]
        print(spec.wave[0].shape,spec.data[0].shape)
        
        # Plotting spectra
        ax1.plot(spec.wave[0],spec.data[0])
        ax1.set_title('Extracted star with sky subtracted')
        ax2.plot(back_spec.wave[0],back_spec.data[0])
        ax2.set_title('Extracted background sky')
        spec.write("spec_without_sky" + "_20_{:03d}.fits".format(i)) # Saving objects spectra.
        back_spec.write("back_spec" + "_20_{:03d}.fits".format(i), overwrite= True) # Saving sky spectra.
    else :
        print('no peak found for slit: ',i)

looking for peaks using 38 pixels around 1108, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 38 pixels around 1173, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 38 pixels around 1371, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 38 pixels around 1561, threshhold of 10.000000
peaks:  [21]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 38 pixels around 1700, threshhold of 10.000000
peaks:  [16]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)


# Checking the accuracy of wavelength calibratio

In [64]:
f2_1 = fits.open('back_spec_20_004.fits')
skyline_file_path = 'pyvista/python/pyvista/data/sky/skyline.dat'
skyline_wavelengths = np.loadtxt(skyline_file_path)

plt.figure()
plt.plot(f2_1[4].data[0], f2_1[1].data[0], )
for wl in skyline_wavelengths:
    plt.axvline(wl, color='red', linestyle='--', alpha=0.5)
#plt.ylabel('Intensity')
plt.title('Sky Spectrum and emission lines Correspondence')
plt.grid(True)
#plt.xlim(7100,8533)
plt.legend()


C:\Users\abdul\AppData\Local\Temp\ipykernel_10524\1287300657.py:13: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


# Spectra stacking

In [67]:
# Define file directories.
from astropy.io import fits
f02 = fits.open('spec_without_sky_17_004.fits')
f03 = fits.open('spec_without_sky_18_004.fits')
f03.info()

Filename: spec_without_sky_18_004.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      67   ()      
  1                1 ImageHDU         7   (4096, 1)   float64   
  2  UNCERT        1 ImageHDU         9   (4096, 1)   float64   
  3  BITMASK       1 ImageHDU        10   (4096, 1)   int32 (rescales to uint32)   
  4  WAVE          1 ImageHDU         8   (4096, 2)   float64   


In [68]:
from astropy.nddata import CCDData # Import astropy data class format. 
from ccdproc import Combiner # Import ccdproc combiner for stacking.

In [69]:
#make a ccddata object out of the flux (pixel count) and uncertainty of each spectra you want to combine
ccd_67 = CCDData(data = f02[1].data[0], uncertainty = f02[2].data[0], unit = 'pixel')
ccd_68 = CCDData(data = f03[1].data[0], uncertainty = f03[2].data[0], unit = 'pixel')
#put them all into Combiner (from ccdproc)
spec_67_69 = Combiner([ccd_67, ccd_68])

#then average combine (it is slightly better than median combining)
avg_test_67_69 = spec_67_69.average_combine()

# Plot and compare.
plt.figure()

plt.plot(f02[4].data[0],f02[1].data[0], label= 'SEG3G2_17')
plt.plot(f03[4].data[0],f03[1].data[0], label= 'SEG3G2_18')
plt.plot(f02[4].data[0], avg_test_67_69.data, label = 'Stacked_spectrum')
plt.legend()
plt.xlabel('Angstrom')
plt.ylabel('Pixel count')

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Text(0, 0.5, 'Pixel count')

## Save the stacked spectra.

In [70]:
hdu = fits.PrimaryHDU(avg_test_67_69)
hdul = fits.HDUList([hdu])

# Write to FITS file
hdul.writeto('Stacked_spec.fits', overwrite=True)